In [22]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [23]:
view_rate = pd.read_excel("/content/view_ratings.xlsx",header =1, index_col= 0)
vr_time = pd.DataFrame([],index=view_rate.index[:-1],columns=view_rate.columns[:-1])

시청률 데이터

In [25]:
def get_table(date):
  jisangpa =pd.read_html(("https://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu=1_1&area=01&begin_date={}").format(date))[8].iloc[3:][[2,3]]
  jongpyeon =pd.read_html(("https://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu=2_1&area=01&begin_date={}").format(date))[8].iloc[3:][[2,3]]
  cable=pd.read_html(("https://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu=3_1&area=01&begin_date={}").format(date))[8].iloc[3:][[2,3]]
  table=pd.concat([jisangpa,jongpyeon,cable])
  table.columns= ['program','view_rate']
  vr_list = []
  for vr in table['view_rate']:
    vr_list.append(float(vr))
  table['view_rate']=vr_list
  table=table.groupby(by=['program']).mean()
  table=table.sort_values(by=['view_rate'],ascending=False)
  table=table[:10]
  table['date']=date
  return table

In [27]:
def rename(table):
  reindex=[] 
  for i in range(len(table.index)):
    program = table.index[i]
    if '1부' in program:
      program=program.replace('1부','')
    if '2부' in program:
      program=program.replace('2부','')
    req = requests.get('https://www.google.com/search?q={} 위키백과'.format(program))
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    reindex.append(soup.select("a > h3")[0].text.split('-')[0][:-1])
  table.index=reindex
  return table

In [28]:
new_table= rename(table)

In [29]:
def sf_time(program):
  req = requests.get('https://ko.wikipedia.org/wiki/{}'.format(program))
  html = req.text
  soup = BeautifulSoup(html, 'html.parser')
  find_time = soup.find("table", class_="infobox").findAll('td')
  
  for contents in find_time:
    if (contents.text[:2]=='매주'):
      start_info =contents.text.strip().split("~")[0].split(" ")
      start_time = start_info[-3]+start_info[-2]
      finish_time = contents.text.strip().split("~")[1]
      return start_time,finish_time

In [30]:
def sf_time_error(program):
  req = requests.get('https://ko.wikipedia.org/wiki/{}'.format(program))
  html = req.text
  soup = BeautifulSoup(html, 'html.parser')
  find_time = soup.find("table", class_="infobox").findAll('td')
  
  for contents in find_time:
    if (contents.text[:2]=='매주'):
      return contents.text.strip()

In [31]:
def jjin(date):
  table = get_table(date)
  new_table= rename(table)
  start_time =[]
  finish_time=[]
  for i in range(len(new_table.index)):
    program = new_table.index[i]
    try:
      start_time.append(sf_time(program)[0])
      finish_time.append(sf_time(program)[1])
    except:
      start_time.append(sf_time_error(program))
      finish_time.append('에러')
  new_table['start time']=start_time 
  new_table['finish time']=finish_time
  return new_table


In [34]:
vr_table =pd.DataFrame([])
for date in vr_time.columns:
  date = date.replace('-','')
  vr_table= pd.concat([vr_table,jjin(date)])

